In [ ]:
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, layers, Input
from sklearn.preprocessing import LabelEncoder

In [ ]:
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)
physical_devices = tf.config.experimental.list_physical_devices('GPU')

print("Num GPUs Available: ", len(physical_devices))

In [ ]:
# Load pretrained CNN model
pretrained_model = load_model("best_model_blueberry.h5")

pretrained_model.summary()

In [ ]:
# Load label embeddings
word_embeddings = np.load("Blueberry_Embeddings.npy", allow_pickle=True).item()  # Dictionary with keys

# Extract embeddings and stack into a 2D array
word_embeddings = np.stack([word_embeddings['Good blueberries'].numpy(), 
                                   word_embeddings['Bad blueberries'].numpy()])  # Convert tensors to numpy arrays

# Ensure word_embeddings is a Tensor
word_embeddings_tensor = tf.convert_to_tensor(word_embeddings, dtype=tf.float32)

In [ ]:
# Freeze the pretrained layers
for layer in pretrained_model.layers:
    
    layer.trainable = False

# Get the third-last layer's output
second_last_layer_output = pretrained_model.get_layer(index = -4).output  # Shape: (None, 128)

# Define a new trainable dense layer with 1024 neurons
dense_layer = layers.Dense(1024, activation = 'relu', name="trainable_dense_layer")(second_last_layer_output)  # Shape: (None, 1024)

# Normalize the dense layer output and word embeddings tensor
dense_layer_norm = tf.nn.l2_normalize(dense_layer, axis=1)  # Normalize along the last dimension
word_embeddings_norm = tf.nn.l2_normalize(word_embeddings_tensor, axis=1)

# Compute the dot product with word embeddings
#dot_product = tf.matmul(dense_layer, tf.transpose(word_embeddings_tensor))  # Shape: (None, 2)
dot_product = tf.matmul(dense_layer_norm, tf.transpose(word_embeddings_norm))  # Shape: (None, 2)

# Apply softmax activation to get predictions
predictions = layers.Softmax(name="softmax_predictions")(dot_product)

In [ ]:
# Define the new model
new_model = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model for training
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

new_model.summary()

In [ ]:
# Load PCA data and labels
data_train = np.load('data_train_PCA.npy')  # Assuming shape (n_samples, 32 * 32 * 199)
data_test = np.load('data_test_PCA.npy')

train_labels = np.load('train_labels.npy', allow_pickle=True)  # Labels in integer format
test_labels = np.load('test_labels.npy', allow_pickle=True)

print(data_train.shape, data_test.shape, train_labels.shape, test_labels.shape )

In [ ]:
le= LabelEncoder()
y_train = le.fit_transform(train_labels)

le2= LabelEncoder()
y_test= le2.fit_transform(test_labels)

In [ ]:
print(np.unique(train_labels))
print(np.unique(y_train))

In [ ]:
def augment_data(sample, label):
    
    # Ensure both are the same type
    random_scale = tf.cast(tf.random.uniform([], 0.9, 1.1), dtype=sample.dtype)
    sample = sample * random_scale  # Random scale between 0.9 and 1.1
    
    return sample, label

In [ ]:
# Convert data to TensorFlow dataset

train_dataset = tf.data.Dataset.from_tensor_slices((data_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((data_test, y_test))

# Apply augmentation only on training data
train_dataset = train_dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle, batch, and prefetch
batch_size = 4

train_dataset = train_dataset.shuffle(buffer_size=100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define the callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='Blueberry_Fused_new2.h5',         # Path to save the best model
    monitor='val_accuracy',                       # Metric to monitor
    save_best_only=True,                      # Save only the best model
    save_weights_only=False,                  # Save the entire model (architecture + weights)
    mode='max',                               # Mode 'min' because we want to minimize validation loss
    verbose=1
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',                       # Metric to monitor
    patience=15,                              # Number of epochs to wait before stopping
    restore_best_weights=True                 # Restore model weights from the epoch with the best metric
)

# Train the model with the callbacks
history = new_model.fit(
    train_dataset,
    epochs=50,
    validation_data=test_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
merged_model = load_model("S3FN.h5")

In [ ]:
# Evaluate the model
test_loss, test_accuracy = merged_model.evaluate(test_dataset)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
test_predictions_prob = merged_model.predict(test_dataset)
test_predictions = np.argmax(test_predictions_prob, axis=1)

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, test_predictions, target_names=["Good_Blueberry", "Bad_Blueberry"]))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#confusion matrix
cm = confusion_matrix(y_test, test_predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Good_Blueberry", "Bad_Blueberry"], yticklabels=["Good_Blueberry", "Bad_Blueberry"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()
